In [1]:
import xarray as xr
import rioxarray
import pandas as pd
import geopandas as gpd
import numpy as np
from glob import glob
import h5py
import cartopy
import pyart
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)
# pyplot needed to plot the dataset, but animation only needed much further down.
from matplotlib import animation
%matplotlib inline

# This is needed to display graphics calculated outside of jupyter notebook
from IPython.display import HTML, display

import sys
sys.path.append('~/PlotGallery')
from matplotlibconfig import basic
basic()


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [7]:
dict_hurricanes= {
    'Harvey': {'datetime': [], 'points':[]},
    'Imelda': {'datetime': [], 'points':[]},
    'Florence': {'datetime': [], 'points': []}}

In [8]:
for event in dict_hurricanes.keys():
    fname= glob('tracks/%s/*_pts.shp'%(event))[0]
    pnts= gpd.read_file(fname)
    pnts.DTG= pd.to_datetime(pnts.DTG, format='%Y%m%d%H')
    dict_hurricanes[event]['datetime']= sorted(pnts.DTG.tolist())
    order= np.argsort(pnts.DTG.values)
    dict_hurricanes[event]['points']= list(zip(pnts.LON.values[order], pnts.LAT.values[order]))

In [9]:
AOI= gpd.read_file('area/great_extent.shp').to_crs('EPSG:4326')

In [17]:
from retrieve_SMAP import read_SML3P_AM, read_SML3P_PM, find_nearest

# Harvey

In [10]:
EVENT= 'Harvey'
fnames= ['/media/scratch/ZhiLi/soil_moisture/CYGNSS/ucar_cu_cygnss_sm_v1_2018_%03d.nc'%dt.dayofyear for dt in dict_hurricanes[EVENT]['datetime']]
cygnss= xr.open_mfdataset(fnames)
cygnss['lat']= cygnss.latitude.values[0,:,0]
cygnss['lon']= cygnss.longitude.values[0,0,:]
cygnss= cygnss.rename({'lon': 'x', 'lat':'y'}).SM_subdaily.rio.set_crs('EPSG:4326').rio.clip([AOI.geometry.iloc[0]])
# ilons= (cygnss.lon<= -74) & (cygnss.lon>= -107)

cygnss_arr= np.zeros((cygnss.shape[0]*4,cygnss.shape[-2],cygnss.shape[-1]))
for i in range(cygnss.shape[0]*4):
    cygnss_arr[i,:,:]= cygnss[i//4,i%4,:,:]

In [ ]:
sms= []
times= []
time_range_SMAP= pd.date_range(pd.to_datetime(dict_hurricanes[EVENT]['datetime'][0]),
                              pd.to_datetime(dict_hurricanes[EVENT]['datetime'][-1]),
                               freq='D')
for datetime in time_range_SMAP:
    fname= glob('/media/scratch/ZhiLi/SMAP/SMAP_L3_SM_P_E_%s*'%datetime.strftime('%Y%m%d'))[0]
    sm,sm_time,lon,lat=read_SML3P_AM(fname)
    
    if ((lon>cygnss.x.values.min()) & (lon<cygnss.x.values.max()) & (lat>cygnss.y.values.min()) &\
        (lat<cygnss.y.values.max())).any():
        sm_values= find_nearest(sm, lon, lat, cygnss.x.values, cygnss.y.values)
        sms.append(sm_values)
        times.append(sm_time)
    sm,time,lon,lat=read_SML3P_PM(fname)
    if ((lon>cygnss.x.values.min()) & (lon<cygnss.x.values.max()) & (lat>cygnss.y.values.min()) &
        (lat<cygnss.y.values.max())).any():    
        
        sm_values= find_nearest(sm, lon, lat, cygnss.x.values, cygnss.y.values)
        sms.append(sm_values)
        times.append(time)    